In [ ]:
# 1. Gerekli paketleri yükleyin
!pip install -q efficientnet_pytorch transformers Pillow

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/s eta 0:00:00


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.2 MB/s eta 0:00:00


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import files
import os

In [ ]:
# 2. NIH ChestX-ray14 etiketleri
CLASSES = [
    "Atelectasis", "Cardiomegaly", "Effusion", "Infiltration",
    "Mass", "Nodule", "Pneumonia", "Pneumothorax",
    "Consolidation", "Edema", "Emphysema", "Fibrosis",
    "Pleural_Thickening", "Hernia", "No Finding"
]

In [ ]:
# rapor şablonu
def generate_professional_report(pred_labels, probs, class_names, patient_info=None, exam_date=None):
    import datetime
    normal_findings = []
    abnormal_findings = []

    pred_labels_set = set([l.replace(" ", "").replace("_", "").lower() for l in pred_labels])

    for idx, cname in enumerate(class_names):
        cname_norm = cname.replace(" ", "").replace("_", "").lower()
        if cname_norm in pred_labels_set:
            abnormal_findings.append((cname, probs[idx]))
        else:
            normal_findings.append((cname, probs[idx]))


    no_finding_norm = "nofinding"
    if (no_finding_norm in pred_labels_set) and (len(pred_labels_set) == 1):
        today = datetime.date.today()
        header = "TORAKS RADYOGRAFİ RAPORU\n\n"
        if patient_info:
            header += f"Hasta Bilgileri: {patient_info}\n"
        if exam_date:
            header += f"Çekim Tarihi: {exam_date}\n"
        else:
            header += f"Rapor Tarihi: {today.strftime('%Y-%m-%d')}\n"
        header += "\n"
        teknik = "Teknik: PA ve lateral akciğer grafisi değerlendirilmiştir.\n\n"
        parankim = "Akciğer Parankimi:\n- Akciğer parankiminde patolojik opasite izlenmemektedir.\n"
        plevra = "\nPlevra:\n- Plevra yapıları doğaldır.\n"
        mediastinum = "\nMediastinum ve Kalp:\n- Kalp konturları ve mediastinal yapılar normal sınırlardadır.\n"
        diyafram = "\nDiyafram ve Kostalar:\n- Diyafram ve kostalarda patolojik bulgu izlenmemektedir.\n"
        sonuc = "\nSonuç:\n- Patolojik bulgu saptanmamıştır.\n"
        sonuc += "\nÖneri:\n- Klinik ve laboratuvar bulgular ile birlikte değerlendirilmesi önerilir.\n"
        return header + teknik + parankim + plevra + mediastinum + diyafram + sonuc


    today = datetime.date.today()
    header = "TORAKS RADYOGRAFİ RAPORU\n\n"
    if patient_info:
        header += f"Hasta Bilgileri: {patient_info}\n"
    if exam_date:
        header += f"Çekim Tarihi: {exam_date}\n"
    else:
        header += f"Rapor Tarihi: {today.strftime('%Y-%m-%d')}\n"
    header += "\n"

    teknik = "Teknik: PA ve lateral akciğer grafisi değerlendirilmiştir.\n\n"

    # Akciğer parankimi
    parankim = "Akciğer Parankimi:\n"
    if any(f[0] in ["Infiltration", "Pneumonia"] for f in abnormal_findings):
        parankim += "- Akciğer alanlarında infiltrasyon ve pnömoni ile uyumlu opasiteler izlenmektedir.\n"
    if any(f[0] == "Nodule" for f in abnormal_findings):
        parankim += "- Nodüler opasiteler dikkati çekmektedir.\n"
    if any(f[0] == "Fibrosis" for f in abnormal_findings):
        parankim += "- Fibrotik değişiklikler mevcuttur.\n"
    if any(f[0] == "Edema" for f in abnormal_findings):
        parankim += "- Artmış interstisyel opasiteler ve ödem bulguları izlenmektedir.\n"
    if not any(f[0] in ["Infiltration", "Pneumonia", "Nodule", "Fibrosis", "Edema"] for f in abnormal_findings):
        parankim += "- Akciğer parankiminde patolojik opasite izlenmemektedir.\n"

    # Plevra
    plevra = "\nPlevra:\n"
    if any(f[0] == "Effusion" for f in abnormal_findings):
        plevra += "- Plevral efüzyon izlenmektedir.\n"
    if any("pleural" in f[0].lower() for f in abnormal_findings):
        plevra += "- Plevral kalınlaşma mevcuttur.\n"
    else:
        plevra += "- Plevra yapıları doğaldır.\n"

    # Mediastinum ve kalp
    mediastinum = "\nMediastinum ve Kalp:\n"
    if any(f[0] == "Cardiomegaly" for f in abnormal_findings):
        mediastinum += "- Kardiyomegali izlenmektedir.\n"
    else:
        mediastinum += "- Kalp konturları ve mediastinal yapılar normal sınırlardadır.\n"

    # Diyafram ve kostalar
    diyafram = "\nDiyafram ve Kostalar:\n- Diyafram ve kostalarda patolojik bulgu izlenmemektedir.\n"

    # Sonuç
    sonuc = "\nSonuç:\n"
    if abnormal_findings:
        bulgular = [f"{f[0]} (olasılık: {f[1]:.2f})" for f in abnormal_findings if f[0] != "No Finding"]
        if bulgular:
            sonuc += "- " + ", ".join(bulgular) + " ile uyumlu bulgular izlenmiştir.\n"
        if any(f[0] == "No Finding" for f in abnormal_findings):
            sonuc += "- Ek olarak, No Finding etiketi de pozitif saptanmıştır. Klinik uyum önerilir.\n"
    else:
        sonuc += "- Patolojik bulgu saptanmamıştır.\n"

    sonuc += "\nÖneri:\n- Klinik ve laboratuvar bulgular ile birlikte değerlendirilmesi, gerekirse ileri tetkik (BT/HRCT) önerilir.\n"

    report = header + teknik + parankim + plevra + mediastinum + diyafram + sonuc
    return report

In [ ]:
# 3. EfficientNet-b0 yükleme
def load_efficientnet_model(weight_path, num_classes=15, device='cpu'):
    model = EfficientNet.from_name('efficientnet-b0')
    model._fc = torch.nn.Linear(model._fc.in_features, num_classes)
    model.load_state_dict(torch.load(weight_path, map_location=device))
    model.eval()
    return model

In [ ]:
# 4. Görsel işleme
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
    return input_tensor

In [ ]:
# 5. Tahmin fonk
def predict_labels(model, input_tensor, threshold=0.5, device='cpu'):
    input_tensor = input_tensor.to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.sigmoid(outputs)[0].cpu().numpy()
    pred_labels = [CLASSES[i] for i, p in enumerate(probs) if p > threshold]
    return pred_labels, probs

In [ ]:
# eğitilmiş modeli yükleme
model_filename = '/content/drive/MyDrive/biochexeff-torch-nih/efficientnet_b0_nih_chestxray_val07834.pth'
if not os.path.exists(model_filename):
    raise Exception("Model dosyası bulunamadı!")

In [ ]:
print("Bir röntgen görseli (.jpg, .png, .bmp) yükleyin:")
uploaded_img = files.upload()
image_filename = None
for fn in uploaded_img.keys():
    if fn.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_filename = fn
        break
if not image_filename:
    raise Exception("Görüntü dosyası yüklenmedi!")

Bir röntgen görseli (.jpg, .png, .bmp) yükleyin:


Saving effusion.jpeg to effusion.jpeg


In [ ]:
# 8. rapor üretimi
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Model yükleniyor...")
model = load_efficientnet_model(model_filename, num_classes=len(CLASSES), device=device)
print("Görsel işleniyor...")
input_tensor = preprocess_image(image_filename)
print("Tahmin yapılıyor...")
detected_labels, probs = predict_labels(model, input_tensor, threshold=0.5, device=device)
print(image_filename)

Model yükleniyor...
Görsel işleniyor...
Tahmin yapılıyor...
effusion.jpeg


In [ ]:
# 9. rapor oluşturma
import datetime
patient_info = "Ad Soyad: Örnek Hasta, Yaş: 60"     # İsterseniz kullanıcıdan da alınabilir
exam_date = datetime.date.today().strftime('%Y-%m-%d')
prof_report = generate_professional_report(detected_labels, probs, CLASSES, patient_info=patient_info, exam_date=exam_date)

print(prof_report)

TORAKS RADYOGRAFİ RAPORU

Hasta Bilgileri: Ad Soyad: Örnek Hasta, Yaş: 60
Çekim Tarihi: 2025-05-28

Teknik: PA ve lateral akciğer grafisi değerlendirilmiştir.

Akciğer Parankimi:
- Akciğer parankiminde patolojik opasite izlenmemektedir.

Plevra:
- Plevral efüzyon izlenmektedir.
- Plevral kalınlaşma mevcuttur.

Mediastinum ve Kalp:
- Kalp konturları ve mediastinal yapılar normal sınırlardadır.

Diyafram ve Kostalar:
- Diyafram ve kostalarda patolojik bulgu izlenmemektedir.

Sonuç:
- Atelectasis (olasılık: 0.60), Effusion (olasılık: 0.79), Mass (olasılık: 1.00), Pleural_Thickening (olasılık: 0.59) ile uyumlu bulgular izlenmiştir.

Öneri:
- Klinik ve laboratuvar bulgular ile birlikte değerlendirilmesi, gerekirse ileri tetkik (BT/HRCT) önerilir.

